# Salary Impact on Competitiveness in Sport

This project can be found on GitHub:

Repository: https://github.com/davehereman/cw2-salary-impact.git

#### Import Libraries

In [1]:
#
# Import libraries
#

# To check existance of files
import os.path

# Import Requests library for getting the information from the webpage
import requests

# Import the Beautiful Soup library | for navigating the webpage
from bs4 import BeautifulSoup

# Read static data input files
import pandas as pd

# Used for statistical calculations
import numpy as np

# Plotting functions
import matplotlib.pyplot as plt
from   matplotlib        import rc
from   mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

# Not sure it this is Seaborn of Matplotlib
#plt.style.use("ggplot") # This defines an aesthetic style


# Import seaborn
import seaborn as sns
# Apply the default theme
#sns.set_theme()

In [2]:
# Define exception classes to help identify issues during design phase

class MissingDataFile(Exception):
    pass


## 3.1 Data Loading
Data loading includings scaping the source websites for data, writing the data to files, reloading it into a Beautiful Soup object, parsing to obtain the requred data, then writing it to Pandas data frames.

#### Salary and Standings Year Range Dictionaries 
Create two dictionaries to hold the range of year to extract for salary and standings for each league. This provides a compact method of entering the data.

In [32]:
# List of leaques for which data is being acquired
leagues = ["epl", "ger", "itl", "spn", "fra", "mls", "mlb", "nfl", "nhl", "nba"]

# Create a list of leagues and corresponding range of years for which data is available
# These list represent the scope of data used in the study

total_salary_league_years = {
    "epl": (2014, 2022),
    "mls": (2018, 2022),
    "mlb": (2020, 2022),
    "nfl": (2020, 2022),
    "nhl": (2020, 2022),
    "nba": (2020, 2022)
}
average_salary_league_years = {
    "epl": (2015, 2019),
    "ger": (2015, 2019),
    "itl": (2015, 2019),
    "spn": (2015, 2019),
    "fra": (2015, 2019),
    "mls": (2015, 2019),
    "mlb": (2015, 2019),
    "nfl": (2015, 2019),
    "nhl": (2015, 2019),
    "nba": (2015, 2019)
}
standings_league_years = {
    "epl": (2013, 2022),
    "ger": (2013, 2022),
    "itl": (2013, 2022),
    "spn": (2013, 2022),
    "fra": (2013, 2022),
    "mls": (2013, 2022),
    "mlb": (2013, 2022),
    "nfl": (2013, 2022),
    "nhl": (2013, 2022),
    "nba": (2013, 2022)
}

# Simple functions to ensure consist naming of output files for scraped data
def total_salary_file(league, year): return "data\TOTAL_SALARY_" + league + "_" + str(year) + ".csv"
def standings_file(league, year): return "data\STANDINGS_" + league + "_" + str(year) + ".csv"
def avg_salary_file(league, year): raise FunctionNotComplete("function still under construction")


#### Read Salary and Standings Paths
Salary and standings paths are saved in the salary_paths.csv and standing_paths.csv file. They are read from the files and saved in a data frame. This expands the year range to a list of years for each league and data set. E.g. epl: 2023-2022 becomes a list of twenty paths.

In [15]:
#
# Read the paths for salary and standings websites from csv files
#

sal_f = "total_salary_data_paths.csv"
std_f = "standings_data_paths.csv"

total_salary_paths = pd.read_csv(sal_f, index_col=["League", "Year"])
standings_paths = pd.read_csv(std_f, index_col=["League", "Year"])
#print(salary_paths)
#print(standings_paths)


# Copy the paths into new data frames, but only for the league-years which are in scope
# The files containing the paths may contain extra league-years. These will be ignored.

total_salary_scope = pd.DataFrame(columns=["League", "Year", "Path"])
total_salary_scope.set_index(keys=["League", "Year"], inplace=True)

standings_scope = pd.DataFrame(columns=["League", "Year", "Path"])
standings_scope.set_index(keys=["League", "Year"], inplace=True)

for l, years in total_salary_league_years.items():
    y0, yn = years
    print("League:", l, "From year:", y0, "To year:", yn)

    for y in range(y0, yn+1):
        print(salary_paths.loc[(l, y), "Path"])
        total_salary_scope.loc[(l, y), "Path"] = total_salary_paths.loc[(l, y), "Path"]

for l, years in standings_league_years.items():
    y0, yn = years
    print("League:", l, "From year:", y0, "To year:", yn)

    for y in range(y0, yn+1):
        print(standings_paths.loc[(l, y), "Path"])
        standings_scope.loc[(l, y), "Path"] = standings_paths.loc[(l, y), "Path"]

print(total_salary_scope)
print(standings_scope)

League: epl From year: 2014 To year: 2022
https://www.spotrac.com/epl/payroll/2014/
https://www.spotrac.com/epl/payroll/2015/
https://www.spotrac.com/epl/payroll/2016/
https://www.spotrac.com/epl/payroll/2017/
https://www.spotrac.com/epl/payroll/2018/
https://www.spotrac.com/epl/payroll/2019/
https://www.spotrac.com/epl/payroll/2020/
https://www.spotrac.com/epl/payroll/2021/
https://www.spotrac.com/epl/payroll/2022/
League: mls From year: 2018 To year: 2022
https://www.spotrac.com/mls/cap/2018/
https://www.spotrac.com/mls/cap/2019/
https://www.spotrac.com/mls/cap/2020/
https://www.spotrac.com/mls/cap/2021/
https://www.spotrac.com/mls/cap/2022/
League: mlb From year: 2020 To year: 2022
https://www.spotrac.com/mlb/payroll/2020/positional/
https://www.spotrac.com/mlb/payroll/2021/positional/
https://www.spotrac.com/mlb/payroll/2022/positional/
League: nfl From year: 2020 To year: 2022
https://www.spotrac.com/nfl/cap/2020
https://www.spotrac.com/nfl/cap/2021
https://www.spotrac.com/nfl/cap

### Scape the required webpages for data and load to BeautifulSoup Objects
Scrape data from the required websites and store as text files, to avoid repeatedly scraping the same sites.

#### scrape()
This function scrapes the target website for the specified data set and writes the data to a file for later processing. It includes only basic error checking as the function is expected to be run only once for each data set.

In [21]:
# Scrape a webpage into a soup object and return the soup

def scrape(URL):
    
    request_header = """
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.50"
    }
    """

    # Scape the website for the requested data

    req = requests.get(URL, request_header)

    if req.status_code != 200:
        print("ERROR Website returned non 200 status code:", req.status_code)
        return None
    
    soup = BeautifulSoup(req.content, "html.parser")
    
    return soup

#### Call scape() for each webpage
To avoid scraping the same webpage multiples times, the function does not scrape if the output file exists.

I used encoding utf-8-sig instead of utf-8. When opening csv files saved as utf-8 using MS Excel, foreign language character became corrupted. utf-8-sig resolved this issue.

The data from each page is saved in the data folder. I have retained only the MLS salary files in this folder. Therefore scraping will skip the corresponding web pages for these files, but scrape all other web pages. A copy of all the files have been saved in the data_copy folder. This is ignored by the function and was saved only for reference. Copying the files into the data sub folder will cause the function to skip the files. Deleting all the file from the data folder will force the function to rescrape the web pages.

In [18]:
#
# Call scrape() for each data set in scope to scrape the from the corresponding web page, 
# then write the data to a file. To avoid repeating scrapes, check for 
# the existance of the output file and only scrape the web page 
# if the file does not exist
#

# To scrape data from all 16 leagues x 20 years
# Run time: 2 min (approx.)
# Data Volume: 70 MB (approx.)

# To avoid scraping websites, copy any/all csv files from data_copy folder to data folder
# Pages for a leauge-year will not be scraped if the target file exists in the data folder


def scrape_wrapper(df, data_type):

    print("\nCommencing scraping web pages - max completion time 2 min (approx.)...\n")
    
    for league_year, data in df.iterrows():

        print("League_Year:", league_year, "Path:", data["Path"])

        # Construct a standard file name based on the data type, leauge, and year
        if data_type == "salary":
            f = salary_file(*league_year)
        elif data_type == "standings":
            f = standings_file(*league_year)

        # Check if salary file exists and skip scraping and writing if it does
        if os.path.exists(f):
            print("File:", f, "already exists")
        else:
            soup = scrape(data["Path"])

            print("Writing:", data_type, "to file:", f, "..........................") # Highlight the message
            with open(f, "x", encoding="utf-8-sig") as target:
                target.write(str(soup))
                
    print("\n           *** COMPLETE *** All required files have been scraped\n")

# Salary and standings URL are saved in separate files and data frames, 
# so call the scaping wrapper funciont once for each data frame
scrape_wrapper(total_salary_scope, "salary")
scrape_wrapper(standings_scope, "standings")



Commencing scraping web pages - max completion time 2 min (approx.)...

League_Year: ('epl', 2014) Path: https://www.spotrac.com/epl/payroll/2014/
File: data\TOTAL_SALARY_epl_2014.csv already exists
League_Year: ('epl', 2015) Path: https://www.spotrac.com/epl/payroll/2015/
File: data\TOTAL_SALARY_epl_2015.csv already exists
League_Year: ('epl', 2016) Path: https://www.spotrac.com/epl/payroll/2016/
File: data\TOTAL_SALARY_epl_2016.csv already exists
League_Year: ('epl', 2017) Path: https://www.spotrac.com/epl/payroll/2017/
File: data\TOTAL_SALARY_epl_2017.csv already exists
League_Year: ('epl', 2018) Path: https://www.spotrac.com/epl/payroll/2018/
File: data\TOTAL_SALARY_epl_2018.csv already exists
League_Year: ('epl', 2019) Path: https://www.spotrac.com/epl/payroll/2019/
File: data\TOTAL_SALARY_epl_2019.csv already exists
League_Year: ('epl', 2020) Path: https://www.spotrac.com/epl/payroll/2020/
File: data\TOTAL_SALARY_epl_2020.csv already exists
League_Year: ('epl', 2021) Path: https

In [27]:
#soup = scrape(data["Path"])

#soup = scrape("https://www.espn.com/soccer/standings/_/league/ger.1/season/2018")
soup = scrape("https://www.espn.com/soccer/standings/_/league/GER.1/season/2018")
#               https://www.espn.com/soccer/standings/_/league/ger.1/season/2018
#                https://www.espn.com/soccer/standings/_/league/GER.1/season/2018
#soup = scrape("https://www.spotrac.com/epl/payroll/2014/")

In [28]:
soup

<!DOCTYPE html >

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]-->
<!--[if IE 9]>    <html class="no-js ie9 oldie" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<!-- start:global -->
<meta charset="utf-8"/>
<!-- end:global -->
<!-- start:page title -->
<title>EPL 2014-2015 Payroll Wages Tracker | Spotrac</title>
<!-- end:page title -->
<!-- start:meta info -->
<meta content="2014-2015,EPL, team payrolls, transfer fees, positional spending, salaries, wages" name="keywords">
<meta content="A real-time look at the payroll totals for each 2014-2015 EPL team, including breakdowns by position." name="description"/>
<meta content="EPL 2014 Payroll Wages Tracker" property="og:title">
<meta content="Spotrac.com" property="og:site_name"/>
<meta content="https://d1dglpr230r5

### Load the saved data into BeautifulSoup objects

### load_data_set() function
This function reads the required data set from file, loads it into a BeautifulSoup object, and returns the object. The data was previously scaped from the source webpage and writen to a file using the scrape() function.

Call the function for each available combination of league and year, and stored the soups in a pd.Series.
Separate calls are made for salary and standings data as they are obtained from different websites/webpages.

In [29]:
# Open the file specified by the data_set argument, and return a BeautifulSoup object

def load_data_set(f):

    with open(f, "r", encoding="utf-8-sig") as file:
        data_text = ""
        for line in file:
            data_text = data_text + line

    #print("[load_data_set]SUCCESS Loaded data from file:", f)
    return BeautifulSoup(data_text, "html.parser")


In [34]:
#
# Call load_data_set() for each data set in scope to load it into a BeautifulSoup object
#
# Run time to load all salary and standings data: 30 seconds (approx.)


# Create two series to hold the soup objects read back from csv files

idx = pd.MultiIndex.from_tuples([], names=["League", "Year"])
salary = pd.Series(index=idx)
standings = pd.Series(index=idx)

print("\n   *** Commence loading files. Run time: 30 sec (approx.)...\n")

# Iterate through all the paths required for salary
print("Loading data from salary files: ", end="")
for i, (league_year, data) in enumerate(total_salary_scope.iterrows()):
    salary[league_year] = load_data_set(total_salary_file(*league_year))
    print(".", end="")
print("\nLoaded", i, "salary files")

# Iterate through all the paths required for standings
print("Loading data from standings files: ", end="")
for i, (league_year, data) in enumerate(standings_scope.iterrows()):
    standings[league_year] = load_data_set(standings_file(*league_year))
    print(".", end="")
print("\nLoaded", i, "standings files")

print("\n   *** Loading complete *** \n")



   *** Commence loading files. Run time: 30 sec (approx.)...

Loading data from salary files: ..........................
Loaded 25 salary files
Loading data from standings files: ................................................................................
Loaded 79 standings files

   *** Loading complete *** 



In [40]:
#('epl', 2015)
standings["fra",2015]

From: <saved blink="" by="">
Snapshot-Content-Location: https://www.espn.com/soccer/standings/_/league/FRA.1/season/2015
Subject: 2015-16 French Ligue 1 French Ligue 1 2015/2016 French Ligue 1 Standings - ESPN
Date: Sat, 19 Aug 2023 07:21:23 -0000
MIME-Version: 1.0
Content-Type: multipart/related;
	type="text/html";
	boundary="----MultipartBoundary--y00Hvy7mpaAjZ9T0qBCWF01gJt1sOhyMJwNvmkGpZo----"


------MultipartBoundary--y00Hvy7mpaAjZ9T0qBCWF01gJt1sOhyMJwNvmkGpZo----
Content-Type: text/html
Content-ID: <frame-5f1a13dbb831413baf0dfe14c2309d9c@mhtml.blink>
Content-Transfer-Encoding: quoted-printable
Content-Location: https://www.espn.com/soccer/standings/_/league/FRA.1/season/2015

<!DOCTYPE html>
<html data-react-helmet='3D"lang"' lang='3D"en"'><head><meta charset="utf-8" content='3D"text/html;' htt='p-equiv=3D"Content-Type"'/><link f1911b88d26@mhtml.blink"="" href='3D"cid:css-1fc17d3a-a055-43a7-accb-5=' rel='=3D"stylesheet"' type='3D"text/css"'/><link href='=3D"cid:css-fa0b7bac-c395-

### Extract MLS salary data from soup into a pandas data frame

Parse the salary soups to extract the required data. Define a function to parse one soup then call the function for each available soup. The final salary data is saved in the salary_df data frame.

The function extracts the team, team code (where available), and the salary. Team code is later recalculated from a single mapping data. The values obtained from scraping salary were used to help construct that mapping table.

In [41]:
import pandas as pd

# Simple function to convert salary given as string to numeric amount
def salary_str_to_float(salary_s):
    salary_s = salary_s.replace("$", "").replace("£", "").replace(",", "")
    return float(salary_s)


# This function parses a soup object and returns a data frame
#    source website: sportrac.com
#
#    league = The league to which the soup data refers
#    soup = soup object holding data previously scraped from a webpage
#    

def extract_salary(league, year, soup):
    
    teams, salaries, codes = [], [], []

    if league in ["mls", "epl", "nfl", "mlb", "nhl", "nba"]:
        
        row = soup.find("div", {"class": "teams"})
        row = row.find("table")
        row = row.find_next("tbody")
        table_rows = row.find_all("tr")
        
        for row in table_rows:
            
            print(".", end="")
            
            elements = row.find_all("td")

            if league == "mls":
                # Team is in column 1. Salary is in column 6
                team = elements[1].string
                salary = elements[6].string
                salary = salary_str_to_float(salary)
                
                code = "TEMP"

            if league == "epl":
                # Team is in column 1. Salary is in column 8
                team = elements[1].string
                salary = elements[8].string
                salary = salary_str_to_float(salary)

                # Trim the team to allow easier mapping
                team = team.replace(" FC", "")
                team = team.replace(" F.C.", "")
                team = team.replace(" A.F.C.", "")
                team = team.replace(".", "")
                
                code = "TEMP"
                
            if league == "nfl":
                # Team and code are in column 1 within span tag
                tr = elements[1]
                tr = tr.find_next("span")
                team = tr.string
                tr = tr.find_next("span")
                code = tr.string
                
                # Total Salary Cap is in column 7 within a span tab
                sr = elements[7]
                salary = sr.find_next("span").string
                salary = salary_str_to_float(salary)
                
            if league == "mlb":
                # Team and code are in column 1 within span tag
                tr = elements[1]
                team = tr.find_next("a").string
                
                # Total Salary Cap is in column 9 within a span tab
                salary = elements[9].string
                salary = salary_str_to_float(salary)
                
                code = "TEMP"
                
            if league == "nhl":
                # Team and code are in column 1 within span tag
                tr = elements[1]
                tr = tr.find_next("span")
                team = tr.string
                code = tr.find_next("span").string
                
                # Total Salary Cap is in column 8 within a span tab
                salary = elements[8].string
                salary = salary_str_to_float(salary)
                
            if league == "nba":
                # Team and code are in column 1 within contents of span tag
                tr = elements[1]
                tr = tr.find("span")
                team = tr.contents[2].strip()
                
                # Total Salary Cap is in column 5 within a span tab
                salary = elements[5].string
                salary = salary_str_to_float(salary)
                
                code = "TEMP"
                
            else:
                pass
            
            # Append the new valuesto the lists within the for loop
            teams.append(team.strip())
            codes.append(code)
            salaries.append(salary)

    # Finally, create a temporary data frame with the extracted data and return the data frame
    df = pd.DataFrame({"League": league, "Year": year, "Code": codes, "Team": teams, "Salary": salaries})

    print()
    return df


In [42]:
#
# Iterate through soups in salary calling extract_salary()
# then append the data frame returned from each call to salary_df data frame
#

salary_df = pd.DataFrame()

for league_year in salary.index:
    
    print("Extracting for league-year:", league_year, end="")
    
    salary_df = pd.concat([
        salary_df,
        extract_salary(*league_year, salary[league_year])
    ])

# If required, can add MultiIndex at this point

print(salary_df)

salary_df.to_csv("salary_dataframe.csv", encoding="utf-8-sig")


Extracting for league-year: ('epl', 2014)..............................................
Extracting for league-year: ('epl', 2015)..............................................
Extracting for league-year: ('epl', 2016)..............................................
Extracting for league-year: ('epl', 2017)......................
Extracting for league-year: ('epl', 2018)......................................
Extracting for league-year: ('epl', 2019).........................................
Extracting for league-year: ('epl', 2020).........................................
Extracting for league-year: ('epl', 2021).........................................
Extracting for league-year: ('epl', 2022).........................................
Extracting for league-year: ('mls', 2018)........................
Extracting for league-year: ('mls', 2019).........................
Extracting for league-year: ('mls', 2020)..........................
Extracting for league-year: ('mls', 2021)..................

### Extract MLS data from soup into to pandas  data frame
As for salary, a function is defined to parse a single soup, then called for each available soup with standings data.

#### Function to extract the MLS standings from the soup object
Each league stores slightly different standings data. To avoid multiple conditional statements as were used for salary, this function uses a dictionary to map target variables to the correct column in the web page's table. This approach is more maintainable and readable.

In [46]:
#
# This dictionary includes the source column number for each var required from the table
# and the target vars which hold the values read from the table. Each league has a dict.
#
source_columns = {
    "mlb": {"wn": 0, "ls": 1},
    "nfl": {"wn": 0, "dw": 2, "ls": 1},
    "nhl": {"wn": 1, "ls": 2, "dw": 3, "pts": 4},
    "nba": {"wn": 0, "ls": 1},
    "mls": {"wn": 1, "ls": 3, "dw": 2, "pts": 7},
    "epl": {"wn": 1, "ls": 3, "dw": 2, "pts": 7},
    "spn": {"wn": 1, "ls": 3, "dw": 2, "pts": 7},
    "ger": {"wn": 1, "ls": 3, "dw": 2, "pts": 7},
    "itl": {"wn": 1, "ls": 3, "dw": 2, "pts": 7},
    "fra": {"wn": 1, "ls": 3, "dw": 2, "pts": 7}
}


In [47]:
# This functions extracts the EPL standings data from a soup object and returns a data frame
#    league = The league to extract for
#    year   = The year to extract
#    soup   = The soup object containing the parsed data from the web page

import pandas as pd
from bs4 import BeautifulSoup

def extract_espn_standings(league, year, soup):

    codes, teams, standings, played, wins, losses, draws, points = [], [], [], [], [], [], [], []

    # Locate the table on the page with the standings for the Regular season
    tables = soup.find_all("table")
    
    # Get the team names and codes from the first table on the page
    # These are consistently identified by span tag with class show-mobile and hide-mobile
    
    rows = tables[0].find_all("tr", attrs={"data-idx": True})

    for r in rows:
        
        # On the MLS page the header row also includes the data-idx attribute, therefore need
        # to explicitly identify and skip the heading row
        if "subgroup-headers" in r["class"]: continue
        
        sp = r.find("span", attrs={"class": "show-mobile"})
        codes.append(sp.string)

        sp = r.find("span", attrs={"class": "hide-mobile"})
        teams.append(sp.string)
     
    # Retrieve the standings data from the second table on the page which contains the standings
    
    rows = tables[1].find_all("tr", attrs={"data-idx": True})
   
    # Use the mapping table in the previous cell to map vars to columns in each web page
    # Initialise the vars as not all columns are applicable to all leagues
    vars = {"wn": 0, "dw": 0, "ls": 0, "pld": 0, "pts": 0}
    
    # Iterate through the rows in the table

    for std, r in enumerate(rows, start=1):
        
        # Explict check to skip header rows for MLS which has two conferences in one page
        # Both conferences are in one table, but standings starts from 1 for each conference
        if "subgroup-headers" in r["class"]: continue
        
        spans = r.find_all("span")   
        
        # Read the available values for the league from the source column based on the map for the league
        # This approach avoids multiple conditional statements

        for col in source_columns[league]:
            vars[col] = int(spans[source_columns[league][col]].string)

        # Append the values for the row to the list for each var
        standings.append(std)
        wins.append(vars["wn"])
        losses.append(vars["ls"])
        draws.append(vars["dw"])
        played.append(vars["wn"] + vars["dw"] + vars["ls"])
        points.append(vars["pts"])

    # Create a new data frame to temporarily hold the data then return the data frame
    df = pd.DataFrame( {
        "League": league,
        "Year": year,
        "Code": codes,
        "Team": teams,
        "Standing": standings,
        "Played": played,
        "Wins": wins,
        "Losses": losses,
        "Draws": draws,
        "Points": points
    })
    
    # For MLS, resort the table because standings from two conferences have been combined
    # For all other US league, divisions and converences were available in a single merged table
    
    if league == "mls":
        df = df.sort_values(by=["Points", "Wins", "Draws"], ignore_index=True, ascending=False)
        df["Standing"] = df.index + 1
    
    return df

#### Call the function to extract standings data for all leagues

In [48]:
#
# Call extract_mls_standings()/extract_epl_standings() for each available combination of leage and year
# Iterate through the combinations and concenate the data frames which are returned by each function.
# Finanaly, merge with the salary data which was extracted from different web pages
#


print("Extracting standings data for all league-years...", end="")

standings_df = pd.DataFrame()

for league, year in standings.index:

    standings_df = pd.concat([
        standings_df, 
        extract_espn_standings(league, year, standings[(league, year)])
    ])
    
    print(".", end="")

standings_df.to_csv("standings_dataframe.csv", encoding="utf-8-sig")

print()
print(standings_df)

Extracting standings data for all league-years...................................

IndexError: list index out of range

In [52]:
soup

<!DOCTYPE html >

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]-->
<!--[if IE 9]>    <html class="no-js ie9 oldie" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<!-- start:global -->
<meta charset="utf-8"/>
<!-- end:global -->
<!-- start:page title -->
<title>EPL 2014-2015 Payroll Wages Tracker | Spotrac</title>
<!-- end:page title -->
<!-- start:meta info -->
<meta content="2014-2015,EPL, team payrolls, transfer fees, positional spending, salaries, wages" name="keywords">
<meta content="A real-time look at the payroll totals for each 2014-2015 EPL team, including breakdowns by position." name="description"/>
<meta content="EPL 2014 Payroll Wages Tracker" property="og:title">
<meta content="Spotrac.com" property="og:site_name"/>
<meta content="https://d1dglpr230r5

# LoadFootball Results Data
Load the results data for all football leagues from 2022 to 2013

## Approach
Data is saved in Excel files. For European leagues (EPL, GER, ITL, SPN, FRA) file exists per league per year. All MLS results are saved in a single file for years 2023 to 2012. Only year 2022 to 2013 are required for this study.

In [156]:
def read_results():
    """
    Read the match results from multiple files and saved them in a data frame."""

    cols = ["League", "Year", "Date", "Time", "H_Code", "H_Team", "A_Code", "A_Team", "H_Goals", "A_Goals", "Result"]
    resdf1 = pd.DataFrame(columns = cols)

    source_cols_pre_2019 = ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR"]
    source_cols = ["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR"]
    source_cols_mls = ["Season", "Date", "Time", "Home", "Away", "HG", "AG", "Res"]
    
    col_map = {
        "HomeTeam": "H_Team",
        "AwayTeam": "A_Team",
        "FTHG": "H_Goals",
        "FTAG": "A_Goals",
        "FTR": "Result"
    }
    col_map_mls = {
        "Season": "Year",
        "Home": "H_Team",
        "Away": "A_Team",
        "HG": "H_Goals",
        "AG": "A_Goals",
        "Res": "Result"
    }

    # Iterate through the leagues and years and store the results in a dataframe.

    for league in ["epl", "ger", "itl", "spn", "fra"]:

        for year in range(2013, 2023):

            results_f = "results/" + league + str(year) + ".csv"
            print("Reading league:", league, "Year:", year, "from:", results_f, "...")

            try:
                if year <= 2019:
                    rdf = pd.read_csv(results_f, header=0, usecols=source_cols_pre_2019)
                else:
                    rdf = pd.read_csv(results_f, header=0, usecols=source_cols)
            except:
                print("[read_results]encountered error opening results file:", results_f)
                pass

            # Rename the available columns and poplate remaining columns
            rdf.rename(columns=col_map, inplace=True)
            rdf["League"] = league
            rdf["Year"] = year
            
            #Assertions - Check that the file contains the correct year's data
            year_d = int("20" + rdf.at[0, "Date"][-2:])
            if year_d != year: raise AssertionError("File year:", year, "does not match data year:", year_d)

            # Append the results to the main results data frame
            resdf1 = pd.concat([resdf1, rdf], ignore_index=True, axis=0, copy=False)

    # MLS results are in a single file combining all years
    
    results_f = "results/mls2023-2012.csv"
    print("Reading league: mls from:", results_f)

    try:
        rdf = pd.read_csv(results_f, header=0, usecols=source_cols_mls)
    except:
        print("[read_results]encountered error opening results file:", results_f)

    # Rename and poplate remaining columns
    rdf.rename(columns=col_map_mls, inplace=True)
    rdf["League"] = "mls"

    # Append the MLS results to the main results data frame
    resdf1 = pd.concat([resdf1, rdf], ignore_index=True, axis=0, copy=False)
            
    return resdf1



In [158]:
# Function to load the team code map

def read_team_code_map(print_map=False):
    """
    This function reads the team code map from a csv file and creates a map.
    The map is used to map team name to team code. This is necessary because different
    data sources use different versions of team names. Mapping multiple different team
    names to a single, consistent team code ensures that data can be combined and analysed.
    
     To use the map, use this notation:
       team_map.at[league, "Code"][team_name]
     Example:
       team_code = team_map.at["epl", "Code"]["Arsenal"]
    """
    
    # Other variations which work and do not work
    #test_var = team_map.at["epl", "Code"]["Arsenal"]                     # works BEST
    #test_var = team_map.loc[("epl", "Arsenal")].at[("epl","Arsenal"), "Code"] # works
    #test_var = team_map.loc[("epl", "Arsenal"), "Code"][0]               # works
    #test_var = team_map["Code"][("epl", "Arsenal")][0]                   # works
    #test_var = team_map.loc["epl", "Code"]["Arsenal"]                    # works
    #test_var = team_map["epl", "Arsenal"]["Code"]                        # does not work


    team_map_f = "team_codes.csv"
 
    try:
        team_map = pd.read_csv(team_map_f, index_col=["League", "Team"])
    except:
        print("Encountered error reading team code map from:", team_map_f)
        return None
    
    if print_map: print(team_map)
    
    
    ##TODO
    # Implement duplicate check
    
    return team_map.sort_index()       # Sorting eliminates a pandas warning.


team_map = read_team_code_map(print_map=True)


                                      Code
League Team                               
epl    Arsenal                         ARS
       Arsenal FC                      ARS
       Aston Villa                     AVL
       Aston Villa FC                  AVL
       Brighton                        BHA
...                                    ...
spn    Real Valladolid                 VLL
       Real Valladolid Club de Fútbol  VLL
       Valladolid                      VLL
       Xerez                           XER
       Real Zaragoza                   ZAR

[619 rows x 1 columns]


In [179]:
# Map the home and away team names to a code using the map team_map. This is a multiindex data frame
def map_results_teams(results_source, test_run = True):
    """
    This functions takes a results data frame and maps the home and away team names to their
    respecitve code. It returns an updated copy of the original data frame.
    
    Setting test_run=True causes the function to terminate early - after 100 records.
    """

    results = results_source.copy()
    
    # For testing purpose, create a list of unmapped teams.
    unmapped_teams = set()
    n_rows = len(results_source)

    for idx, mr in results.iterrows():
        #print(idx, mr["League"], mr["H_Team"], mr["A_Team"])

        # Map both the home and away teams to their respective codes. Update the corresponding columns
        for team_col, code_col in [("H_Team", "H_Code"), ("A_Team", "A_Code")]:
            
            #print("team_col:", team_col, "team_code:", code_col)
            #print("leage:", mr["League"], "team:", mr[team_col])
                
            #print("DEBUG Team map epl, Arsenal:", team_map.loc["epl", "Code"]["Arsenal"])

            try:
                code = team_map.loc[mr["League"], "Code"][mr[team_col]]
                results.at[idx, code_col] = code
                #print("Team:", mr[team_col], "New code:", code)
                
            except:
                unmapped_teams.add((idx, mr["League"], mr["Year"], mr[team_col]))
                #print("Unmapped Team:", mr[team_col])
                
                #DEBUG

        if idx % 100 == 0:
            print(".", end="")
        if idx % 1000 ==0:
            print(int(idx / n_rows * 100), "%", end="")
        if idx >= 100 and test_run:
            print("WARNING Test run. Breaking after 100 records")
            break

    if not test_run: print("\n\n-----------Mapping complete.----------\n")
    print(unmapped_teams)
    return results, unmapped_teams

In [180]:
# Read the results from the collection of csv files
#res1 = read_results()
print("Number of match results read:", len(res1))
#res1.to_csv("res1_dataframe.csv")

team_map = read_team_code_map(print_map=False)
res2, unmapped_teams = map_results_teams(res1, test_run=False)
res2.to_csv("res2_dataframe.csv")




Number of match results read: 22728
.0 %..........4 %..........8 %..........13 %..........17 %..........21 %..........26 %..........30 %..........35 %..........39 %..........43 %..........48 %..........52 %..........57 %..........61 %..........65 %..........70 %..........74 %..........79 %..........83 %..........87 %..........92 %..........96 %.......

-----------Mapping complete.----------

set()


In [205]:
import unittest

class TestResultsData(unittest.TestCase):
    """
    This test case performs data quality check on the results data after it has been loaded to 
    a pandas data frame.
    """
    
    def test_year(self):
        years = set(res2["Year"])
        targ_years = set([y for y in range(2012, 2024)])
       
        self.assertEqual(years, targ_years)
        

    
###### print(res2.shape)
#print(res2.columns.values)
#print(res2.iloc[0, 5])
#print(print("Team type:", type(res2.at[0,"H_Team"])))
#print(print("Code type:", type(res2.at[0,"H_Code"])))


# Check if any match have the same codes for the home and away teams

#print(res2.loc[res2["H_Code"] == "ARS"])
#print(dup_team_codes)

#dup_df = res2.loc[0:18161]

#print(dup_df)
#print(dup_df["H_Team"]=="Arsenal")
#print(dup_df["H_Code"]=="ARS")
#b_list = dup_df["H_Team"]=="Arsenal"
#c_list = dup_df["H_Code"]=="ARS"

#print("c_list == c_list:\n", b_list==c_list)

#print(">", "ARS", "<",sep="")
#print(">", dup_df.loc[0,"H_Code"], "<", sep="")

#dup_df.loc[c_list]
#dup_slice = res2.loc[res2["H_Code"]==res2["A_Code"]]


#print(dup_df.loc[dup_df["H_Code"]=="ARS"])
#dup_df = dup_df.loc[dup_df["League"]=="epl"]
#dup_df = dup_df.loc[dup_df["Year"]==2013]
#dup_df = dup_df.loc[dup_df["Date"]=="17/08/13"]
#dup_df = dup_df.loc[dup_df["Time"]==np.nan]
#dup_df = dup_df.loc[dup_df["H_Team"]=="Arsenal"]
#dup_df = dup_df.loc[dup_df["A_Team"]=="Aston Villa"]
#dup_df = dup_df.loc[dup_df["A_Code"]=="AVL"]
#dup_df

unittest.main(defaultTest=["TestResultsData"], argv=['ingored', '-v'], exit=False)    


test_year (__main__.TestResultsData.test_year) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


,League,Year,Date,Time,H_Code,H_Team,A_Code,A_Team,H_Goals,A_Goals,Result


In [168]:
print(team_map.loc["mls", "Code"]["DC United"])
print(team_map.loc["itl", "Code"]["Genoa"])
print(team_map.loc["epl", "Code"]["Arsenal"])
print(team_map)

DC
GEN
ARS
                          Code
League Team                   
epl    AFC Bournemouth     BOU
       AFC Bournemouth FC  BOU
       Arsenal             ARS
       Arsenal FC          ARS
       Aston Villa         AVL
...                        ...
spn    Valladolid          VLL
       Vallecano           RAY
       Villarreal          VIL
       Villarreal CF       VIL
       Xerez               XER

[619 rows x 1 columns]


In [201]:
a = [a for a in range(1,10)]
a


[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [20]:
class dffunc:
   
    def __init__(self):
        self.const = 2
        
    def map(self, a):
        return a * self.const
    
    def print_const(self):
        print(self.const)
    
test_cls = dffunc()
test_cls.print_const()

test_map = {2: "two", 0: "zero"}



testdf2 = pd.DataFrame(columns = ["first","sec","thir", "for"])
testdf1 = pd.DataFrame([[1,2,3], [0,0,0]], columns = ["first","sec","thir"])
#testdf2 = pd.DataFrame([[4,6,7,8], [8,9,0,7]], columns = ["first","sec","thir", "for"])


testdf3 = pd.concat([testdf1, testdf2], ignore_index=True)
print(testdf1)
print(testdf2)

print(testdf3)
print(len(testdf3))

col_map = {"first": "The one"}

testdf3.rename(columns=col_map, inplace=True)
print(testdf3)

testdf3["thir"] = testdf3["thir"].apply(test_cls.map)
testdf3["sec"] = testdf3["sec"].apply(lambda x: test_map[x])
print(testdf3)

testdf3.loc[testdf3["The one"]==testdf3["sec"]]

print("2123"=="2123")



2
   first  sec  thir
0      1    2     3
1      0    0     0
Empty DataFrame
Columns: [first, sec, thir, for]
Index: []
  first sec thir  for
0     1   2    3  NaN
1     0   0    0  NaN
2
  The one sec thir  for
0       1   2    3  NaN
1       0   0    0  NaN
  The one   sec  thir  for
0       1   two     6  NaN
1       0  zero     0  NaN
True


In [101]:
if True: raise AssertionError("Year of file does not year of data")

AssertionError: Year of file does not year of data

In [355]:
testset = set((1,2,3))

testset.add(4)
print(testset.)

AttributeError: 'set' object has no attribute 'size'